To formalize the problem, we want to arrange the numbers 1–14 in a sequence $x_1,x_2,\dots,x_n$ such that for every number in the sequence $x_i$, $|x_{i-1}\pm x_i|$ and $|x_i\pm x_{i+1}|$ are prime, and the sequence wraps around at the edges.

While this problem is small enough to solve by hand, my first instinct was to use the puzzle as an introduction for how _easy_ it is to solve logic puzzles out-of-the-box with Prolog — a constraint logic programming language. This post will examine the above puzzle along with two additional games to showcase how Prolog allows for intuitive solutions to nontrivial problems.

## Circular primes

### Logic primality tests and Prolog introduction

Since we'll be dealing with primes, we'll need a way to identify primes. As Prolog is a logic programming language, it doesn't actually have _functions_, but _predicates_ — statements about some input(s) that either hold or don't hold. Here's [one way of many](https://stackoverflow.com/questions/23282097/prolog-program-to-check-if-a-number-is-prime) we can check for primality in Prolog: 

```prolog
is_prime(2) :- !.
is_prime(3) :- !.

is_prime(N) :-
  N > 3,
  N mod 2 =\= 0,
  is_prime(N, 3).
is_prime(N, X) :-
  ( X*X > N ->
    true
  ; N mod X =\= 0,
    M is X + 2,
    is_prime(N, M)
  ).
```

The goal of this post is not to teach Prolog, but demonstrate it — however, some background is still necessary, so the following will hopefully serve as a whirlwind tour of Prolog. Firstly, there are two main ways of interfacing with Prolog: facts/rules, and queries. Facts and rules determine what can be queried, and they define expressions either declaratively or implicitly, respectively. For example, `married(bob, john).` is a fact declaring Bob to be married to John. A rule could be `is_single(Person) :- not(married(Person, _) ; married(_, Person)).` which effectively says a person is single if they are not married to someone else (we need to account for both positions in the `married` relation).

To explain the above notation, predicates are defined (usually in snake case) with the syntax `head :- body.` which reads as "the predicate `head` holds if `body` holds." The body of a predicate consists of clauses linked by conjunction "and" (with a comma `,`), disjunction "or" (with a semicolon `;`), or ended by a period `.`. Variables always begin with uppercases, and atoms (constants) start with a lowercase. When we write `is_prime(2) :- !.`, we are asserting that `is_prime` holds for the number two, and for three. This is almost equivalent to `is_prime(2).`, which is valid Prolog, but the `!` adds a subtle difference which we'll get to shortly.

Prolog's inference engine features backtracking, which means it will generate multiple pathways for an execution to follow, pursue one, and then follow-up on any paths it can still potentially yield a satisfactory answer to. For example, the `is_single` predicate "matches" any `married` facts where `Person` is in either position (the underscore `_` means the other person can be anyone).

Querying in Prolog lets you interact with the "database" you've built from facts and rules/predicates. For example, if we query

```prolog
?- is_single(john).
false.
```

we see that John is married to someone. If we try an undefined name (atom), we have:

```prolog
?- is_single(mary).
true.
```

So how do we do things with variables in Prolog? Well -- this is where it gets interesting. Let's say we want to find who is married to whom. Since we know John isn't single, let's find out:

```prolog
?- married(john, Person).
false.
```

Whoops — order matters in Prolog, and we wrote our fact as `married(bob, john).`, so we need to check for both patterns:

```prolog
?- married(john, Person) ; married(Person, john).
Person = bob.
```

The result shows us what variables were "bound" to make the query true. You can equivalently think of it as saying "here's what needs to be true for the query to hold." What if there are multiple ways to satisfy a query? Let's add another marriage for John -- `married(john, sam).` -- and see:

```prolog
?- married(john, P) ; married(P, john).
P = sam ;
P = bob.
```

Here, Prolog finds multiple solutions to satisfy the query, so we are prompted with the choice of either terminating the search (by pressing the period key) or entering a semicolon to find further answers.

That leaves us with `!`. Simply put, the bang character in Prolog says "if you've reached here and evaluated true, stop searching." You can also think of it as Prolog's `break` statement. Let's try inserting a bang in the above query:

```prolog
?- married(john, P), ! ; married(P, john).
P = sam.
```

Since we matched `married(john, sam).` first, Prolog continues the conjunction with `!` and stops search, returning the first-found variable to bind.

Hopefully this is enough to understand the primality test code above -- the only other syntactical quirks to point out is that `( X -> Y ; Z )` is an if-then-else block (`if X then Y else Z`), and predicates can share names as long as they have different *arity* (number of predicate arguments), so `is_prime` can be defined twice and not clash.

### Translating satisfiable conditions

Now that we can test for prime numbers, we need a method of determining whether the sum and difference of two numbers are primes:

```prolog
square_sumdiff(N, M) :-
  Sum is N + M,
  Diff is abs(N - M),
  is_prime(Sum),
  is_prime(Diff).
```

This code is pretty self-explanatory. We take in two numbers `N` and `M`, compute the sum and difference (in Prolog, `is` evaluates an expression), and ensure both are prime.

Next, we need a way of iteratively placing numbers in a "circle" -- we'll use a list and place our numbers left-to-right. Without worrying about efficiency (appending and swapping, etc.) the problem can be defined recursively:

* **Base case**: we've placed all of our numbers correctly except for one -- this last number must "fit" with $x_{n-1}$ and $x_1$
* **Recursive case**: we need to place some number $x_i$ next to $x_{i-1}$ which satisfies the sum-difference rule, and then place the rest of the remaining numbers

Let's try to translate these cases into a predicate `place_adjacents`, starting with the base case:

```prolog
place_adjacents([N], Placements, [First|Acc]) :-
  last(Acc, Last),
  square_sumdiff(Last, N),
  square_sumdiff(N, First),
  append([First|Acc], [N], Placements).
```

In the predicate signature, we use Prolog's pattern matching to say that the first argument must be a singleton list (with the last number `N` to place), `Placements` is the final placement list, and `[First|Acc]` is the list so far with the first element `First` at the head of the list of placements, and `Acc` is everything else accumulated after the first element (in Prolog, the pipe character `|` is the construct operator).

The body of the predicate simply extracts $x_{n-1}$, comparing $x_i$ to it and $x_{1}$, ensuring the sums and differences are primes. Then, we concatenate the list so far with the last element to get the final list of placements, `Placements`.

What about the recursive case?

```prolog
place_adjacents(Nums, Placements, Acc) :-
  select(N, Nums, Nums1),
  last(Acc, Last),
  square_sumdiff(Last, N),
  append(Acc, [N], Acc1),
  place_adjacents(Nums1, Placements, Acc1).
```

This is similar to the base case, but more general -- now we use the built-in function to `select` an element `N` from the available numbers `Nums`, leaving us with one less number in `Nums1`. Then we compare this `N` with $x_{i-1}$ and add it to our accumulator if compatible. If all of these steps hold, we must also ensure that the placement of the rest of the numbers holds, too -- so we call the same predicate on the pruned number list and new accumulator.

Note that `select` is actually nondeterministic here — we don't know which number Prolog is going to take out of the list to succeed, as the predicate may only hold with certain numbers. Additionally, I should note that I'm describing the code iteratively, as if walking through each clause in the predicate. However, as a logic programming language, the more natural interpretation would be to say that *every clause* must hold in the predicate (even though order does matter in Prolog) for execution to succeed.

### Bringing it all together

Now that we've established predicates that hold when every number is placed correctly in a list, we need to generate the `Nums` list to hold our numbers. You may have also noticed we made the assumption in the above predicates that $x_{i-1}$ always exists (i.e. the list is not empty) — so we need to begin with a number in the list (it doesn't matter what we start with, since the list "wraps around" itself).

```prolog
circular_primes(Low, High, Placements) :-
  Low < High,
  bagof(N, between(Low, High, N), Nums),
  select(First, Nums, NewNums),
  place_adjacents(NewNums, Placements, [First]).
```

In the predicate `circular_primes`, we take in a lower and upper bound such that $\forall i:x_i\in[\verb|Low|,\verb|High|]$. After confirming the range is valid, we use `bagof` to take all `N` such that `N` is between `Low` and `High`, storing the list in `Nums`. Then we take the first element `First` to get all the other numbers `NewNums`, and state that in order for `circular_primes` to hold, we must have a placement of all other numbers `NewNums` with the first number that satisfies the constraints of the problem.

Now we can solve the riddle! All that's left is to enter query mode and ask Prolog:

```prolog
?- circular_primes(1, 14, Ps).
Ps = [1, 4, 7, 10, 13, 6, 11, 8, 3, 14, 9, 2, 5, 12] .
```

With the numbers 1–14, the predicate succeeds with the above circular list of numbers. Feel free to validate the solution yourself.

The fun part is, now that we've written a general solver for this type of question, we can ask Prolog for any range of numbers:

```prolog
?- circular_primes(6, 24, Ps).
false.

?- circular_primes(6, 23, Ps).
Ps = [6, 11, 8, 15, 22, 19, 12, 7, 10, 21, 16, 13, 18, 23, 20, 9, 14, 17] .
```